In [ ]:

# This program is meant to help build features 
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from path import Path
from os import listdir
import os
from os.path import isfile, join
import shutil
from scipy import signal
import sys
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_relevant_features


noise = -92

class Peak:
    def __init__(self, start, end, height):
        self.start = start
        self.end = end
        self.width = end - start
        self.height = height
    def getStart(self):
        return self.start
    def getEnd(self):
        return self.end
    def getWidth(self):
        return self.width
    def getHeight(self):
        return self.height               
    def __str__(self):
        return "START: " + str(self.start) + "; END: " + str(self.end) + "; WIDTH: " + str(self.width) + "; HEIGHT: " + str(self.height)
    def __repr__(self):
        return str(self) + '\n'

def amplitude(val):
    global noise
    if val == '0j' or val == '' or val == '0':
        return noise
    comp = complex(val)
    real = comp.real  #extract real part and convert to a float
    imaginary = comp.imag #remove the space between the sign and number for imaginary and convert to float
    
    refLevDBM = 30
    absComplex = abs(complex(real, imaginary))
    
    try:
        sigDB = 20* math.log(absComplex, 10)
        
    except:
        print("val is " + str(val))
        print("abscomplex is " + str(absComplex))
        print(sigDB)
        sys.exit()
        
    sigDBM = sigDB + 30 - refLevDBM
    
    return sigDBM #complex takes two floats, real and imaginary, and creates a complex number, return that complex number




       
def whiskerStDev(df, peak):
     # this method is meant to find the standard deviation of between 
    # the top and bottom end of the whisker. To achieve this, I will 
    # find the center of the peak, and look for the max and min of the 
    # center left and center right of the whisker
    # samples around the center
    
    start = peak.getStart()
    end = peak.getEnd()
    
    center = (start + end) // 2
    
    centerLeft = (center + start)//2
    centerRight = (center + end)//2
    
    stdev = df['Amplitude'][centerLeft : centerRight].std()
    
    if math.isnan(stdev):
        return 0
    return stdev
    
    
    

    
def whiskerHeight(df, peak):
    # this method is meant to find the difference between the top 
    # and bottom end of the whisker. To achieve this, I will find 
    # the center of the peak, and look for the max and min of the 
    # center left and center right of the whisker
    # samples around the center

    start = peak.getStart()
    end = peak.getEnd()
    
    center = (start + end) // 2
    
    centerLeft = (center + start)//2
    centerRight = (center + end)//2
    
    
    maximum = df['Amplitude'][centerLeft : centerRight].max()
    minimum = df['Amplitude'][centerLeft : centerRight].min()
    if math.isnan(maximum) or math.isnan(minimum):
        return 0
    return maximum - minimum
    

# what I want to do is write something which tells me when the slope has suddenly changed....
# alternatively I could have something which tells me when the mean amplitude has changed,
# tells me the exact location for the change if the average is above noise for 4000 samples
# add start end and width to possible peaks
def identifyPeaks(data):
    df = data.copy()
    
    numSamples = df.index.max()
    jumpBy = 1000
    roll = 500
    df['Amplitude'] = df['Amplitude'].rolling(roll).mean(center = True)
    i = 0
    peaks = []
    
    
    if numSamples < minWidth:
        return []
    
    while i < df.index.max():
        end = 0
        start = 0
        amp = df['Amplitude'][i]
        if amp >= noise:
            while amp >= noise and i >= 0: #looks for peak's exact start
                i-=1
                amp = df['Amplitude'][i]
                
            start = i
            amp = df['Amplitude'][i+1]
            while amp >= noise and i + 100 <= numSamples :
                i+=100
                amp = df['Amplitude'][i]
                
            while amp <= noise and i >= 0:
                i-=1
                amp = df['Amplitude'][i]
                end = i
                
            if end - start >= minWidth and end != 0 and start != roll - 2:
                
                plt.plot(data['Amplitude'][max(0, start - 200) : min(end + 200, numSamples)])
                plt.savefig(savePlotsTo + '//' + device + '//' +str(burstNum)+ "_" + str(len(peaks)))
                plt.cla()
                
                center = (start + end) // 2
                centerLeft = (center + start)//2
                centerRight = (center + end)//2
            
                height = df['Amplitude'][centerLeft: centerRight].mean()
                newPeak = Peak(start, end, height)
                peaks.append(newPeak)
        i+=jumpBy
    
#     print(peaks)
    
    if len(peaks) > 2:
        peaks = twoHighest(peaks)
    
    return peaks

def twoHighest(peaks):
    
    if len(peaks) == 2:
        return peaks
    
    elif len(peaks) == 1:
        zeroPeak = Peak(0, 0, 0)
        peaks.append(zeroPeak)
        return peaks
    
    elif len(peaks) == 0:
        zeroPeak = Peak(0, 0, 0)
        zeroPeak2 = Peak(0, 0, 0)
        peaks.append(zeroPeak)
        peaks.append(zeroPeak2)
        return peaks
    
    else:
        peaks.sort(key = Peak.getHeight)
#         print(peaks)
        peaks = peaks[-2:]
        peaks.sort(key = Peak.getStart)
        return peaks
                
def getSpace(peaks):
    space = peaks[1].start - peaks[0].end
    if space <= 0:
        return 0
    return space
    
def getDeviceName(subdir):
    i = -1
    device = ''
    while abs(i) < len(subdir):
        if not subdir[i] == '\\':
            device += subdir[i]
            i -= 1
        else:
            return device[::-1]
        
def deviceArr(path):
    
    devices = []
    
    my_directory = Path(path)
    subdirs = my_directory.dirs()
    
    i = 0
    for subdir in subdirs:
        devices.insert(i, getDeviceName(subdir))
        i+=1
    return devices

def makeDir(pathWay):
    if os.path.isdir(pathWay):
        shutil.rmtree(pathWay)
        
    os.mkdir(pathWay)
    
def getBurstNum(file):
    i = -1
    num = ''
    
    while abs(i) < len(file):
        if not file[i] == '.':
            i -= 1
            
        else:
            i-=1
            while abs(i) < len(file) and file[i] != 't': 
                num += file[i]
                i-=1
                    
            return int(num[::-1])
        
        
def timeSeries(mainSeries):
    extracted_features = extract_features(timeseries, column_id="Device", column_sort="Time")
    y = mainSeries["Device"]
    impute(extracted_features)
    features_filtered = select_features(extracted_features, y)
    features_filtered_direct = extract_relevant_features(timeseries, y,
                                                     column_id='Device', column_sort='Time')
    features_filtered_direct.to_excel("timeseriesFeatures.xlsx")
    
    
    
    
    
if __name__ == "__main__":
    # assumptions, there is a directory which has folders labelled by device name. The folder contains amplitude information. 
    # requirements: must be able to pick up the two strongest plateaus
    # will use python peak detection to find out the two or one peaks worth looking at. Will look at the most prominent peaks
    # first.
    path = r'Bursts'
    savePlotsTo = r'Plots'
    featuresFolder = r'Features'
    samplingRate = 112*(10**6) # sampling rate of collection device in samples/second
    minLength = 40/(10**6) # min length of a bluetooth packet (8us for preamble and 32us for access address) in seconds
    minWidth = minLength * samplingRate #minwidth is 4480samples
    
    features = pd.DataFrame(columns = ['Bursts', 'Device', 'Burst Width','Burst-Top Height',
                                                          'Burst-Top Stdev'])
    mainseries = pd.DataFrame()
    devices = deviceArr(path)
    
    my_directory = Path(path)
    subdirs = my_directory.dirs()
    
    makeDir(savePlotsTo)
    makeDir(featuresFolder)
    
    
    for device in devices:
        makeDir(savePlotsTo + '//' + device) #make folder to save plots to
        
        print(device)
        subdir = path + '\\' + device
        
        bursts = [f for f in listdir(subdir) if isfile(join(subdir, f))]
        
        size = len(bursts) * 2 # we expect each burst to contain 2 bursts
        
        df = pd.DataFrame(index = range(size), columns = ['Bursts', 'Device', 'Burst Width','Burst-Top Height',
                                                          'Burst-Top Stdev'])
        i = 0
        for file in bursts:
            
            fileDir = path + '\\' + device + '\\' + file
            print(fileDir)
            data = pd.read_excel(fileDir)
            data = data.rename(columns = {0 : 'Amplitude'})
            data['Amplitude'] = data['Amplitude'].apply(amplitude) # convert to amplitude
            data = data.rename(columns = {'Unnamed: 0': 'Time'})
            data['Device'] = device
            
            if len(data['Amplitude']) <= minWidth:
                print("skipped " + fileDir)
                continue
            
            
            
            
            
            burstNum = getBurstNum(file)
            data['Burst'] = burstNum
            
#             plt.plot(data['Amplitude'])
#             plt.savefig(savePlotsTo + '//' + device + '//' +str(burstNum))
#             plt.cla()
            
#             insert into file
            peaks = identifyPeaks(data)
    
            if i + len(peaks) >= size:
                print("resizing... size was " + str(size) + " and is now " + str(size*2) )
                size *= 2
                df = df.reindex(range(size))
                print(df.index.max())
                
            for peak in peaks:
            
                df['Bursts'][i] = burstNum
                df['Device'][i] = device
                df['Burst Width'][i] = peak.getWidth()
                df['Burst-Top Height'][i] = whiskerHeight(data, peak)
                df['Burst-Top Stdev'][i] = whiskerStDev(data, peak)
                timeseries = data[peak.start : peak.end]
                mainseries = mainseries.append(timeseries)
#                 print(mainseries)
#                 print(str(peak) + " " + str(i))
                i+=1
                
            
                
#         print(df)
        df = df.reindex(range(i))
        df.to_excel(featuresFolder + "\\" + device + '.xlsx')
        features = features.append(df)
    timeSeries(mainseries)
        

features = features.reindex(sorted(features.columns), axis=1)
print(features)
features.to_excel(featuresFolder + '\\Features.xlsx')
            
            #smooth the data



Apple watch
Bursts\Apple watch\Burst000.xlsx
Bursts\Apple watch\Burst001.xlsx
Bursts\Apple watch\Burst002.xlsx
Bursts\Apple watch\Burst003.xlsx
Bursts\Apple watch\Burst004.xlsx
Bursts\Apple watch\Burst005.xlsx
Bursts\Apple watch\Burst006.xlsx
Bursts\Apple watch\Burst007.xlsx
Bursts\Apple watch\Burst008.xlsx
Bursts\Apple watch\Burst009.xlsx
Bursts\Apple watch\Burst010.xlsx
Bursts\Apple watch\Burst011.xlsx
Bursts\Apple watch\Burst012.xlsx
Bursts\Apple watch\Burst013.xlsx
Bursts\Apple watch\Burst014.xlsx
Bursts\Apple watch\Burst015.xlsx
Bursts\Apple watch\Burst016.xlsx
Bursts\Apple watch\Burst017.xlsx
Bursts\Apple watch\Burst018.xlsx
Bursts\Apple watch\Burst019.xlsx
Bursts\Apple watch\Burst020.xlsx
Bursts\Apple watch\Burst021.xlsx
Bursts\Apple watch\Burst022.xlsx
Bursts\Apple watch\Burst023.xlsx
Bursts\Apple watch\Burst024.xlsx
Bursts\Apple watch\Burst025.xlsx
Bursts\Apple watch\Burst026.xlsx
Bursts\Apple watch\Burst027.xlsx
Bursts\Apple watch\Burst028.xlsx
Bursts\Apple watch\Burst029.xls